In [ ]:
import cmocean
import geopandas as gpd
import matplotlib.patheffects as pe
import matplotlib.pyplot as plt
import pandas as pd
import rioxarray as rxr

# Coordinates of approx. route
# Via https://www.distance.to/Kairouan/37.081475648860525,11.017317584242392/37.80544394934274,12.456322824638164/38.18638677411551,15.685637729722657/38.89958342598271,16.267793137782064/40.019201307686785,15.663669601116567/40.64730356252251,14.795928521178894/40.79717741518769,14.103932470089376/43.01268088642034,10.47919125009662/44.02442151965934,10.083764935188356/Vignole-Borbera/Geneva
TN_to_CERN = [
    (35.666668, 10.100000, "Kairouan"),
    (37.081475648860525, 11.017317584242392),
    (37.80544394934274, 12.456322824638164),
    (38.18638677411551, 15.685637729722657),
    (38.89958342598271, 16.267793137782064),
    (40.019201307686785, 15.663669601116567),
    (40.64730356252251, 14.795928521178894),
    (40.79717741518769, 14.103932470089376),
    (43.01268088642034, 10.47919125009662),
    (44.02442151965934, 10.083764935188356),
    (44.709949, 8.891260),
    (46.203812, 6.139959, "Geneva"),
]

# via https://www.distance.to/Geneva/Lyon/Avignon/Montpellier/43.08092540794885,3.2181972752839556/42.12267315117256,3.366482143374583/41.520916896362515,2.641533899375986/41.1455697310095,1.644983611941143/40.027614437486655,0.3268958955800372/38.788345355085625,0.30492776697406043/37.57941251343841,-0.7275742775087792/36.63316209558658,-2.3312476657479237/35.94243575255426,-5.428753799196329/Fez
MA_to_CERN = [
    (46.203812, 6.139959, "Geneva"),
    (45.767300, 4.834329),
    (43.948612, 4.805967),
    (43.608177, 3.879446),
    (43.08092540794885, 3.2181972752839556),
    (42.12267315117256, 3.366482143374583),
    (41.520916896362515, 2.641533899375986),
    (41.1455697310095, 1.644983611941143),
    (40.027614437486655, 0.3268958955800372),
    (38.788345355085625, 0.30492776697406043),
    (37.57941251343841, -0.7275742775087792),
    (36.63316209558658, -2.3312476657479237),
    (35.94243575255426, -5.428753799196329),
    (34.043571, -4.995548, "Fez"),
]

# Convert to geopandas DataFrame which is easier to use
MA_to_CERN = gpd.GeoDataFrame(MA_to_CERN, columns=["Latitude", "Longitude", "City"])
TN_to_CERN = gpd.GeoDataFrame(TN_to_CERN, columns=["Latitude", "Longitude", "City"])

# Cities for annotation later in plot
cities = pd.concat([TN_to_CERN.iloc[[0, -1]], MA_to_CERN.iloc[[-1]]])

# Load and downsample bathymetric data for map
gebco = rxr.open_rasterio(snakemake.input["gebco"])
gebco = gebco.rio.write_crs(4326)

# Limits to dataset; will also become limits to plots figure
gebco = gebco.sel(band=1, x=slice(-10, 18), y=slice(48, 30))

from rasterio.enums import Resampling

downscale_factor = 1 / 16

# Caluculate new height and width using downscale_factor
new_width = gebco.rio.width * downscale_factor
new_height = gebco.rio.height * downscale_factor

# downsample raster: sufficient resolution for plotting, more efficient
down_sampled = gebco.rio.reproject(
    gebco.rio.crs,
    shape=(int(new_height), int(new_width)),
    resampling=Resampling.bilinear,
)

# Data for overlay of country borders
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

# Plot
fig, ax = plt.subplots(figsize=(6, 6))

# Bathymetric data
down_sampled.plot(
    vmin=-600,
    vmax=0,
    cmap=cmocean.cm.deep_r,
    # levels=11,
    cbar_kwargs={
        "orientation": "vertical",
        "shrink": 0.5,
        "aspect": 30,
        "label": "Water depth [m]",
        "pad": 0.02,
    },
    ax=ax,
)

# Country borders (based on natural earth)
world.plot(
    color="none",
    edgecolor="black",
    ax=ax,
)

# Plot suggested routes
ax.plot(
    MA_to_CERN["Longitude"],
    MA_to_CERN["Latitude"],
    linestyle="solid",
    color="#A7B480",
    linewidth=3,
)

ax.plot(
    MA_to_CERN["Longitude"],
    MA_to_CERN["Latitude"],
    #    marker="o",
    #    markersize=2,
    linestyle="solid",
    color="#FF0000",
    linewidth=2,
)

ax.plot(
    TN_to_CERN["Longitude"],
    TN_to_CERN["Latitude"],
    linestyle="solid",
    color="#A7B480",
    linewidth=4,
)

ax.plot(
    TN_to_CERN["Longitude"],
    TN_to_CERN["Latitude"],
    #    marker="o",
    #    markersize=2,
    linestyle="solid",
    color="#FF0000",
    linewidth=2,
)

# Add city labels
row = cities.iloc[0]
ax.annotate(
    text=row["City"],
    xy=(row["Longitude"], row["Latitude"]),
    xytext=(-8, 0),
    textcoords="offset points",
    ha="right",
    va="center",
    backgroundcolor="#fdfecc",
)
row = cities.iloc[1]
ax.annotate(
    text=row["City"],
    xy=(row["Longitude"], row["Latitude"]),
    xytext=(-7, 7),
    textcoords="offset points",
    ha="right",
    va="center",
)
row = cities.iloc[2]
ax.annotate(
    text=row["City"],
    xy=(row["Longitude"], row["Latitude"]),
    xytext=(25, 0),
    textcoords="offset points",
    ha="right",
    va="center",
    backgroundcolor="#fdfecc",
)

# Add city markers
for idx, row in cities.iterrows():
    ax.scatter(x=row["Longitude"], y=row["Latitude"], marker="x", color="black")

# Make plot nicer
# ylim, xlim are specified via bounds of dataset earlier
# ax.set_xlim(-10,18)
# ax.set_ylim(30,48)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax.set_title("")

fig.tight_layout()

# Save output
fig.savefig(snakemake.output["pdf"], dpi=300)
fig.savefig(snakemake.output["png"], dpi=300)